**************** ETL *****************

El nomenclador utilizado en dataframes:
- df_c dataframe
- df_c dataframe copy
- df_n dataframe nuevo, con columnas seleccionadas
- s_ serie 
- s__ y__n serie nueva, limpia
- _l limpio
- _v partes que vienen de df_visual
- _a partes que vienen de df_actores
- _c partes que vienen de df_categorias

Se importan las librerias necesarias

In [14]:
import pandas as pd


Se importan las funciones que se van creando en paralelo para el ETL en un archivo .py

In [15]:
from funciones_etl import *

Se crea las variables con las paths_ vamos a respetar el orden alfabético de los archivos durante el EDA y ETL

In [16]:
path_amazon="Datasets\\amazon_prime_titles.csv"
path_disney="Datasets\\disney_plus_titles.csv"
path_hulu="Datasets\\hulu_titles.csv"
path_netflix="Datasets\\netflix_titles.json"

Se abren los archivos con la función de apertura creada (luego de realizar el EDA que está en un jupiter aparte) se comienza con el ETL- VER EN README - la estrategia utilizada, la cual esta COMPLETAMENTE adaptada al objetivo de las querys propuestas. 

IMPORTANTE, se creo una función para abrir los archivos, sin embargo no pudo ser aplicada en el docker por problemas de instalacion de chardet

In [17]:
df_amazon=pd.read_csv(path_amazon)
df_disney=pd.read_csv(path_disney)
df_hulu=pd.read_csv(path_hulu)
df_netflix=pd.read_json(path_netflix)

Como buena practica se realiza una copia de cada dataframe 

In [18]:
df_amazon_c=df_amazon.copy()
df_disney_c=df_disney.copy()
df_hulu_c=df_hulu.copy()
df_netflix_c=df_netflix.copy()

NO se eliminan duplicados, porque no tienen:
 
#df_amazon_c.drop_duplicates(inplace=True) 
#df_disney_c.drop_duplicates(inplace=True)
#df_hulu_c.drop_duplicates(inplace=True)
#df_netflix_c.drop_duplicates(inplace=True)

CRITERIO DE LIMPIEZA:

Dado que las querys propuestas requieren, UNICAMENTE de las columnas: 

- title
- type
- cast
- listed_in (categoria)
- release year 
- platform
- duration


Se eliminaron todas las demás columnas: 

Justificacion, el objetivo de este proyecto es hacer la conxecion con la API, la consulta por la misma y levantarla en un docker. No es realizar un datawarehouse estructurado donde podamos realizar cualquier consulta, por lo que el ELT se realizó en base al objetivo planteado. 

Hubiera sido otro el criterio de haber pedido la información disponible para otros usos en un data warehouse. 

Me quedo con las columnas que necesito

In [19]:
df_amazon_new=eliminando_columnas(df_amazon_c)
df_disney_new=eliminando_columnas(df_disney_c)
df_hulu_new=eliminando_columnas(df_hulu_c)
df_netflix_new=eliminando_columnas(df_netflix_c)

A cada Dataframe se le agrega una columna con su fuente

In [20]:
df_amazon_new['platform']='amazon'
df_disney_new['platform']='disney'
df_hulu_new['platform']='hulu'
df_netflix_new['platform']='netflix'

Separamos duration en dos nuevas columnas, una para seasons en las series y otra para tiempo de duracion de las peliculas

In [21]:
df_hulu_new=separando_duration(df_hulu_new)
df_hulu_new.drop(columns=['duration'], inplace = True)
df_amazon_new=separando_duration(df_amazon_new)
df_amazon_new.drop(columns=['duration'], inplace = True)

df_disney_new=separando_duration(df_disney_new)
df_disney_new.drop(columns=['duration'], inplace = True)

df_netflix_new=separando_duration(df_netflix_new)
df_netflix_new.drop(columns=['duration'], inplace = True)


Concatenado de tablas

In [22]:
df_visual=pd.concat([df_amazon_new, df_disney_new,df_hulu_new,df_netflix_new], axis=0)

Se elimina el viejo ID, se hace un eordenamiento de columnas y renombramiento de listed_in como categorías
Se utiliza el mismo index del dataframe como ID global autoincremental.

In [23]:
df_visual.drop(columns=['show_id'], inplace = True)
df_visual= df_visual.rename(columns={df_visual.columns[4]: 'category'})
df_visual = df_visual.reindex(columns=["title","type","category","release_year","dur_time","cast","platform"])

In [24]:
df_visual.isnull().sum()


title               0
type                0
category            0
release_year        0
dur_time        22998
cast             5321
platform            0
dtype: int64

In [25]:
df_visual['cast']=df_visual['cast'].fillna('null')

Checkpoint

In [26]:
df_visual.sample(10)

,title,type,category,release_year,dur_time,cast,platform
7093,Inxeba,Movie,"Dramas, Independent Movies, International Movies",2017,NaN,"Nakhane Touré, Bongile Mantsai, Niza Jay, Thob...",netflix
8725,Act 6 - Series 1,TV Show,Action,2021,NaN,1,amazon
39,Yoga to Harness Your Chakra Energy,Movie,"Faith and Spirituality, Special Interest",2014,NaN,Caroline Shola Arewa,amazon
6093,Dark,Movie,"Action, Suspense",2018,NaN,"Mohanlal, Nadhiya, Parvatii Nair, Nassar",amazon
2479,The Lovebirds,Movie,"Action & Adventure, Comedies, Romantic Movies",2020,NaN,"Issa Rae, Kumail Nanjiani, Paul Sparks, Anna C...",netflix
2775,Hunt for the Wilderpeople,Movie,"Adventure, Comedy, Drama",2016,NaN,null,hulu
618,Rūrangi,TV Show,"Documentaries, International, LGBTQ+",2021,NaN,null,hulu
2590,Early Day Miners,Movie,Music Videos and Concerts,2011,NaN,null,amazon
6412,Halloween Kids Songs by Little Baby Bum,Movie,Kids,2019,NaN,null,amazon
6609,Die Another Day,Movie,Action & Adventure,2002,NaN,"Pierce Brosnan, Halle Berry, Toby Stephens, Ri...",netflix


Se pasa a un csv limpio

In [27]:
filepath = pl('csv_limpios/visualgeneral.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_visual.to_csv(filepath,
                 sep= ",",
                 na_rep= "null",
                 columns= ("title","type","category","release_year","dur_time","cast","platform"),
                 header= True,
                 index= False,
                 index_label= None,
                 mode= "w",
                 encoding= "utf-8",
                 compression= "infer",            
                 #chunksize= None, - es el número de row - puedo limitarlo aca
                 decimal= "."
                 ) 
#df_visual.to_csv(filepath, sep=';', na_rep='sin_dato', columns=None, header=True, index=False)
os.makedirs('csv_limpios', exist_ok=True)  
df_visual.to_csv('csv_limpios/visualgeneral.csv')  